In [ ]:
# scrape_yahoo_v2.ipynb
# 2021.02.07  scrape_yahoo_v2.ipynb
# 2020.07.10  scrape_yahoo_VOO.ipynb

In [56]:
from lxml import html  
import requests
# from exceptions import ValueError
from time import sleep
import json
import argparse
from collections import OrderedDict
# from time import sleep
import time
import pandas as pd
import urllib3
import gc
urllib3.disable_warnings()

def parse_etf(ticker):
    url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    url ="https://finance.yahoo.com/quote/%s?p=%s&.tsrc=fin-srch"%(ticker,ticker)
    response = requests.get(url, verify=False)
#     print( "Parsing %s"%(url))
#     time.sleep(4)
    print( f'Parsing ticker={ticker}')
    parser = html.fromstring(response.text)
    summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    tt = summary_json_response.json()
    
    expRatio = get_ExpRatio(response)
    
    res = [ticker,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
#     tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
    str(round(expRatio*100,2)) +"%"       ,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt'] 
          ]
    return res

def get_ExpRatio(response):
    s = response.text
    s2 = "annualReportExpenseRatio"  #239750
    s2 = "feesExpensesInvestment"    #240971

    n1=s.find(s2)
    # print(n1)
    N=400
    n2=s[n1:(n1+N)].find("annualReportExpenseRatio")
    expr = float(s[n1+n2:(n1+n2+N)].split(":")[2].split(",")[0])
    return expr

def strip_pct_char(col):
    for i in range(len(col)):
        if col[i][-1]=='%':
            col.loc[i] = col[i][:-1]
    return col

In [127]:
# del df
if len(df)>0:
    del df

In [128]:
columns = ['ticker', 'fundFamily', 'category', 'yield', 'annualReportExpenseRatio','beta3Year',  'ytdReturn' ]
tickers = ['VOO', 'CLTL','SHV','VGSH','VGIT','GOVT','IEF','BND', 'JNK', 'ARKW', 'VGT','QQQ','VDE','EDV', 'BLV', 
           'VGLT', 'VMBS',  'VTIP',  'VCIT', 'VCSH', 'VTC', 'VTEB', 
           'VIG',  'VUG', 'VYM', 'BNDW']

BAD_TICKERS = ['BEP','GLDM','LLY']
tickers = ['VOO', 'BND', 'JNK', 'ARKW', 'VGT','QQQ','VDE','VIG',  'VUG', 'VYM', 'BNDW','XBI','TAN']
tickers = ['VOO', 'BND']

res = [parse_etf(ticker=tkr) for tkr in tickers ]


df=pd.DataFrame(res, columns=columns)
df.head()

Parsing ticker=VOO
Parsing ticker=BND


,ticker,fundFamily,category,yield,annualReportExpenseRatio,beta3Year,ytdReturn
0,VOO,Vanguard,Large Blend,1.56%,0.03%,2.09%,1.00
1,BND,Vanguard,Intermediate-Term Bond,2.22%,0.04%,-1.01%,1.03


### Strip '%', rename some cols accordintly, define 'cy', 'ncy' (normalized cy), and make all float

In [129]:
# col='yield'
# df[col] = pd.to_numeric(strip_pct_char( df[col] ))

In [130]:
cols_with_pct = ['yield','annualReportExpenseRatio','beta3Year']
for col in cols_with_pct:
    df[col] = pd.to_numeric(strip_pct_char( df[col] ))
    
df.rename(columns={'yield': 'yield_pct', 
                   'annualReportExpenseRatio': 'annualReportExpenseRatio_pct',
                   'beta3Year':'beta3Year_pct'}, inplace=True)

df['ytdReturn'] =  pd.to_numeric(df['ytdReturn'])

# df['cy_pct'] = [float(df['yield_pct'][k]) - float(df['annualReportExpenseRatio_pct'][k]) for k in range(len(df))]
# df['ncy'] = [round(df['cy_pct'][k]/float(df['beta3Year_pct'][k]),2) if float(df['beta3Year_pct'][k])>0 else -100 for k in range(len(df))]
# print('NOTE: "cy_pct"= cumulative yield = yield_pct - annualExpRatio_pct')
# print('NOTE: "ncy  "= normalized clean yield = cy/beta3Year')

# df = df.sort_values(by=['ncy'],ascending=False).reset_index()
# df = df.drop(['index'], axis=1)
df

C:\Users\khafifa\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,ticker,fundFamily,category,yield_pct,annualReportExpenseRatio_pct,beta3Year_pct,ytdReturn
0,VOO,Vanguard,Large Blend,1.56,0.03,2.09,1.00
1,BND,Vanguard,Intermediate-Term Bond,2.22,0.04,-1.01,1.03


In [99]:
df.columns

Index(['ticker', 'fundFamily', 'category', 'yield_pct',
       'annualReportExpenseRatio_pct', 'beta3Year_pct', 'ytdReturn', 'cy_pct',
       'ncy'],
      dtype='object')

In [100]:
for col in ['yield_pct','annualReportExpenseRatio_pct', 'beta3Year_pct', 'ytdReturn', 'cy_pct','ncy']:
    print( type(   df[col][0]   ))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [101]:
gc.collect()

5706

In [ ]:
# df.to_excel('Vang_ETF_comp.xlsx')

In [102]:
df

,ticker,fundFamily,category,yield_pct,annualReportExpenseRatio_pct,beta3Year_pct,ytdReturn,cy_pct,ncy
0,JNK,SPDR State Street Global Advisors,High Yield Bond,5.13,0.40,0.71,0.62,4.73,6.66
1,VYM,Vanguard,Large Value,3.19,0.06,1.63,0.92,3.13,1.92
2,VOO,Vanguard,Large Blend,1.56,0.03,2.09,1.00,1.53,0.73
3,VDE,Vanguard,Equity Energy,4.53,0.10,11.67,1.97,4.43,0.38
4,VGT,Vanguard,Technology,0.83,0.10,3.00,1.09,0.73,0.24
5,VUG,Vanguard,Large Growth,0.66,0.04,2.92,1.04,0.62,0.21
6,QQQ,Invesco,Large Growth,0.55,0.20,5.25,1.04,0.35,0.07
7,ARKW,ARK ETF Trust,Technology,1.30,0.79,18.63,1.48,0.51,0.03
8,XBI,SPDR State Street Global Advisors,Health,0.20,0.35,15.62,1.28,-0.15,-0.01
9,TAN,Invesco,Miscellaneous Sector,0.09,0.69,12.51,1.53,-0.60,-0.05


In [103]:
import matplotlib.pyplot as plt
import folium
import os
# import json
import numpy as np
import pandas as pd

In [131]:
# plt.rcParams['figure.figsize'] = [ 20, 12 ]
# plt.rcParams.update({'font.size': 18})
# fig, ax = plt.subplots()

# x = df['cy_pct']
# ax.set_xlabel('cy_pct')

# y = df['ytdReturn']
# ax.set_ylabel('ytdReturn')

# # ax.plot(   x, y,'-',color='r',LineWidth=2, label='true f')
# ax.scatter(x, y, s=200, alpha=0.3)


# ax.grid(True)   
# # cmap = get_cmap('tab10')
# # colors = cmap.colors
# # ax.set_prop_cycle(color=colors)
# # ax.legend(loc=legend_loc)
# # # legend = ax.legend(loc='upper right', shadow=True, fontsize='x-large')
# # # # Put a nicer background color on the legend.
# # # legend.get_frame().set_facecolor('C0')
# # [Lr, Xsr, Xer] = np.round([L, Xs, Xe],1)


# # https://stackoverflow.com/questions/14432557/matplotlib-scatter-plot-with-different-text-at-each-data-point
# # for i, txt in enumerate(d.shape[0]):
# for i in range(df.shape[0]):
#     ax.annotate(df['ticker'][i], ( x[i], y[i]))

    
# from datetime import datetime
# ttl_str = datetime.today().strftime('%Y-%m-%d')    
# plt.title( ttl_str )
# plt.show()

In [114]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')
# '2021-01-26'

'2021-02-07'